In [ ]:
#! pip install planet

In [1]:
import os
import requests

import math
import rasterio
import matplotlib.pyplot as plt
import planet as planet

In [2]:
from requests.auth import HTTPBasicAuth

## Planet images

In [ ]:
# The area of interest (AOI) defined as a polygon
usenge_aoi = {
    "type":
    "Polygon",
    "coordinates": [[[34.072946,-0.10027800000000298],
                     [34.099198,-0.10027800000000298],
                     [34.099198,-0.06602800000000286],
                     [34.072946,-0.06602800000000286],
                     [34.072946,-0.10027800000000298]]]
}

# The item IDs we wish to order
usenge_images = ['20220626_070918_80_2442', '20220626_070916_53_2442']

In [ ]:
import geopandas as gpd

gdf = gpd.read_file('~/Documents/Modules/Thesis/lake_victoria/data/Ayanga/explorer-ayanga-aoi.geojson')
gdf.to_file('~/Documents/Modules/Thesis/lake_victoria/data/Ayanga/explorer-ayanga-aoi.shp')

In [ ]:
# Google Earth Engine configuration
cloud_config = planet.order_request.google_earth_engine(
    project='lake-victoria', collection='ayanga-aoi')
# Order delivery configuration
delivery_config = planet.order_request.delivery(cloud_config=cloud_config)

In [ ]:
# Product description for the order request
data_products = [
    planet.order_request.product(item_ids=usenge_images,
                                 #product_bundle='analytic_udm2',
                                 #item_type='PSScene'*/
                                 product_bundle='analytic_udm2',
                                 item_type='PSScene'
                                )
]

# Build the order request
usenge_order = planet.order_request.build_request(name='usenge_order',
                                                products=data_products,
                                                delivery=delivery_config)

print(usenge_order)

In [ ]:
 # Clip images to the AOI's perimeter and harmonize the data with Dove Classic
tools = [
    planet.order_request.clip_tool(iowa_aoi),
    #planet.order_request.harmonize_tool('Sentinel-2')
]

# Build the order request
usenge_order = planet.order_request.build_request(name='ayanga_order',
                                                products=data_products,
                                                delivery=delivery_config,
                                                tools=tools)

print(iowa_order)

In [ ]:
async def create_and_deliver_order(order_request, client):
    '''Create and deliver an order.

    Parameters:
        order_request: An order request
        client: An Order client object
    '''
    with planet.reporting.StateBar(state='creating') as reporter:
        # Place an order to the Orders API
        order = await client.create_order(order_request)
        reporter.update(state='created', order_id=order['id'])
        # Wait while the order is being completed
        await client.wait(order['id'],
                          callback=reporter.update_state,
                          max_attempts=0)

    # Grab the details of the orders
    order_details = await client.get_order(order_id=order['id'])

    return order_details

In [ ]:
async with planet.Session() as ps:
    # The Orders API client
    client = ps.client('orders')
    # Create the order and deliver it to GEE
    order_details = await create_and_deliver_order(usenge_order, client)

### Sindo: Planet data

In [3]:
image_file = "/Users/aida/Documents/Modules/Thesis/lake_victoria/data/Ogal/hamilton/Ogal.tif"
sat_data = rasterio.open(image_file)
sat_data

<open DatasetReader name='/Users/aida/Documents/Modules/Thesis/lake_victoria/data/Ogal/hamilton/Ogal.tif' mode='r'>

In [4]:
width_in_projected_units = sat_data.bounds.right - sat_data.bounds.left
height_in_projected_units = sat_data.bounds.top - sat_data.bounds.bottom

print("Width: {}, Height: {}".format(width_in_projected_units, height_in_projected_units))


Width: 4819.313910000026, Height: 3101.9163300003856


In [5]:
print("Rows: {}, Columns: {}".format(sat_data.height, sat_data.width))

Rows: 57411, Columns: 89197


In [6]:
print(sat_data.count)

# sequence of band indexes
print(sat_data.indexes)

4
(1, 2, 3, 4)


In [7]:
# Load the 4 bands into 2d arrays - recall that we previously learned PlanetScope band order is BGRN.
b, g, r, n = sat_data.read()

In [1]:
# Displaying the blue band.

fig = plt.imshow(b)
plt.show()

NameError: name 'plt' is not defined

In [ ]:
# Displaying the green band.

fig = plt.imshow(g)
fig.set_cmap('gist_earth')
plt.show()

In [ ]:
# Displaying the red band.

fig = plt.imshow(r)
fig.set_cmap('inferno')
plt.colorbar()
plt.show()

In [ ]:
# Displaying the infrared band.

fig = plt.imshow(n)
fig.set_cmap('winter')
plt.colorbar()
plt.show()

## Sindo : Sentinel2 data

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import rasterio
import os
from osgeo import gdal

In [ ]:
fp_in='../data'
fp_out='../data'

data_path = '/Sindo/HomaBayCounty_Sindo_July_10_sentinel2l1c_analytic/files/'

fn_blue='T36MXE_20220710T074619_B02_clip'
fn_green='T36MXE_20220710T074619_B03_clip'
fn_red='T36MXE_20220710T074619_B04_clip'

In [ ]:
band_02=rasterio.open(fp_in+ data_path +fn_blue+'.tif')
band_03=rasterio.open(fp_in+ data_path +fn_green+'.tif')
band_04=rasterio.open(fp_in+ data_path +fn_red+'.tif')

In [ ]:
red = band_04.read(1)
green = band_03.read(1)
blue = band_02.read(1)

In [ ]:
fig = plt.figure(figsize=(20,6))
ax1 = fig.add_subplot(1,3,1)
ax1.imshow(red, cmap='Reds')
ax1 = fig.add_subplot(1,3,2)
ax1.imshow(green, cmap='Greens')
ax1 = fig.add_subplot(1,3,3)
ax1.imshow(blue, cmap='Blues')

In [ ]:
def normalize(band):
    band_min, band_max = (band.min(), band.max())
    return ((band-band_min)/((band_max - band_min)))

red_n = normalize(red)
green_n = normalize(green)
blue_n = normalize(blue)

In [ ]:
rgb_plot=plt.imshow(rgb_composite_n, interpolation='lanczos')
plt.axis('off')


In [ ]:
rgb_composite_n= np.dstack((red_n, green_n, blue_n))
plt.imshow(rgb_composite_n)

In [ ]:
Z = rgb_composite_n

In [ ]:
new_dataset = rasterio.open(
    fp_out + data_path + 'sentinel2_rgb_composite2.tif',
    'w',
    driver='GTiff',
    height=Z.shape[0],
    width=Z.shape[1],
    count=1,
    dtype=Z.dtype,
    crs='+proj=latlong'
)

In [ ]:
rgb_plot=plt.imshow(rgb_composite_n, interpolation='lanczos')
plt.axis('off')
plt.savefig(fp_out+ data_path + 'sentinel2_rgb_composite.png',dpi=200,bbox_inches='tight')
plt.close('all')

In [ ]:
#rasterio.open(fp_in + data_path + 'sentinel2_rgb_composite2.tif')
fp_in + data_path + 'sentinel2_rgb_composite2.tif'

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
# Reading the data inside the xml
# file to a variable under the name
# data
path = '../data/Sindo/HomaBayCounty_Sindo_July_10_sentinel2l1c_analytic/files/S2B_MSIL1C_20220710T074619_N0400_R135_T36MXE_20220710T094014_metadata_aux_clip/'
with open(path + 'MTD_MSIL1C.xml', 'r') as f:
    data = f.read()
 
# Passing the stored data inside
# the beautifulsoup parser, storing
# the returned object
Bs_data = BeautifulSoup(data, "xml")

b_unique = Bs_data.find_all('Radiometric_Offset_List')
print(b_unique)